In [1]:
# Initial settings
%cd ..

import time
import numpy as np
import pandas as pd

C:\Users\vujanicr\PycharmProjects\nsopy-stoch


# Performance Comparison of Dual Methods

## 1. Parameter Selection

In this notebook we run the experiments necessary to determine good parameter settings for the different methods. We only consider the main parameter of each method
* **Subgradient**: stepsize $p = s_0$
* **Universal Gradient Methods**: optimality tolerance $p = \epsilon$
* **Quasi Monotone Methods**: $p = \gamma$
* **Cutting Planes/Bundle Methods**: optimality tolerance $p = \epsilon$

We select one representative instance of each model and test each method with parameter settings $p \in \left\{ 0.1, 1, 10, 100 \right\}$.

In [2]:
#######################
# Experiment Settings #
#######################

# 1) Do restricted number of tests?
# if True, only run tests for one method (TA 2)
process_only_one_solution_method = True
# if True, only test one instance class (sslp)
process_only_one_class = True

# 2) Output files names
filename_parameter_scan_record = 'ZZ_parameter_scan_record.csv'
filename_parameter_selection = 'ZZ_parameter_selection.csv'

# 3) Max time (approx) for single run in minutes
single_experiment_timeout = 0.1  # for the paper we set this to 30 [minutes]

In [3]:
from nsopy import DualMethodsFactory, AVAILABLE_METHODS
from nsopy import EnhancedDualMethodLogger, SlimDualMethodLogger
from nsopy.methods.utils import record_logger
from itertools import product

# Determine EXPERIMENT tuples ('method', parameter)
PARAMETERS = (0.01, 0.1, 1.0, 10.0, 100.0)
if process_only_one_solution_method:
    AVAILABLE_METHODS = ('TA 2',)  # to test only one method
experiments = product(AVAILABLE_METHODS, PARAMETERS)  # iterable
experiments = [exp for exp in experiments]  # make flat list because we want to iterate over it many times

We then pick thet first instance of each problem type as a representative

##### Important: the code has been changed so that only the problem set tested is `SSLP`. Uncomment box to run everything. `SSLP` only takes about an hour on a normal PC; all instances will take 4-6 days.

In [4]:
from smps.inner_problems_factory import BenchmarkInnerProblemsFactory, STOCH_INSTANCES

# run only one class
if process_only_one_class:
    inner_problems = []
    for subtype in STOCH_INSTANCES:
        if subtype == 'sslp':
            inner_problems.append(BenchmarkInnerProblemsFactory(type='2 stage stoch', subtype=subtype, instance_n=0))

# run all tests
elif not process_only_one_class:
    inner_problems = []
    for subtype in STOCH_INSTANCES:
        if subtype != 'all':
            inner_problems.append(BenchmarkInnerProblemsFactory(type='2 stage stoch', subtype=subtype, instance_n=0))

Parsing nominal model information from smps/benchmark_problems/2_sslp/sslp_5_25_3_mymod.cor and .tim ...
Parsing stochastic information from smps/benchmark_problems/2_sslp/sslp_5_25_3_mymod.sto ...
Stochastic model is of type SCENARIOS DISCRETE


In [5]:
methods = [[DualMethodsFactory(ip, method=exp[0], param=exp[1]) for exp in experiments] for ip in inner_problems]
loggers = [[SlimDualMethodLogger(method) for method in ip] for ip in methods]

In [6]:
# Setting appropriate dual domain for bundle and cutting planes methods
for ip in methods:
    for method in ip:
        if 'Cutting' in method.desc or 'Bundle' in method.desc:
            method.set_dual_domain(type='2 stage smps')

Start with the experiments. Method loggers are objects that store the results from each run; the conent of these loggers is then saved into csv files by calling `record_logger()`:

In [7]:
for index_ip, ip in enumerate(methods):
    for index_method, method in enumerate(ip):
        print method
        
        # Determine N_ORACLE_CALLS allocated for experiment
        start_time = time.time()
        method.oracle(method.projection_function(np.zeros(method.dimension, dtype=float)))
        oracle_time = time.time() - start_time

        # Set value
        N_ORACLE_CALLS = min(1000,  single_experiment_timeout*float(60)/oracle_time)
        print 'oracle_time  = {}, N_ORACLE_CALLS = {}'.format(oracle_time, N_ORACLE_CALLS)
        
        i = 0
        while method.oracle_calls < N_ORACLE_CALLS and i < N_ORACLE_CALLS:
            method.dual_step()
            i += 1  # we have to add this because cutting planes stop querying oracles when they find the solution
        
        record_logger(loggers[index_ip][index_method], filename=filename_parameter_scan_record)

oracle_time  = 0.077999830246, N_ORACLE_CALLS = 76.9232443337
File ZZ_parameter_scan_record.csv does not exist
Record file does not exist. Creating ZZ_parameter_scan_record.csv ...
oracle_time  = 0.0550000667572, N_ORACLE_CALLS = 109.09077668
oracle_time  = 0.0599999427795, N_ORACLE_CALLS = 100.000095368
oracle_time  = 0.0540001392365, N_ORACLE_CALLS = 111.110824617
oracle_time  = 0.106000185013, N_ORACLE_CALLS = 56.6036747886


The results are logged in the file `parameter_scan_record.csv`. We determine now the best parameter for each method for each class of instances.

In [8]:
# Read in file and prepare extra columns
results = pd.read_csv(filename_parameter_scan_record)
# add new columns
results['d_k_max'] = None
results['iteration of d_k_max'] = None
results['is_competitive'] = None

Determine "winning" parameter, store it in `parameter_selection` (dataframe) and write results in `parameter_selection.csv`.

In [9]:
from nsopy.methods.utils import flatten_record_dataframe

results = flatten_record_dataframe(results)
parameter_selection = pd.DataFrame(columns=('subtype', 'method_name', 'selected_parameter'))

for subtype in results['instance_subtype'].unique():
    for method_name in results['method_name'].unique():        
        # First pass: find minimum, and competitive parameters (i.e., within 1%) and iteration of minimum
        maxima = []
        maxima_iteration = []
        for index, result in results[(results['instance_subtype']==subtype) & (results['method_name']==method_name)].iterrows():
            results.set_value(index, 'd_k_max', max(result.d_k))
            maxima.append(max(result.d_k))
            results.set_value(index, 'iteration of d_k_max', result.oracle_calls[np.argmax(result.d_k)])
            maxima_iteration.append(np.argmax(result.d_k))

        # Then search for the "winner" parameter
        winner_iterations = np.infty
        for index, result in results[(results['instance_subtype']==subtype) & (results['method_name']==method_name)].iterrows():
            if (abs(result.d_k_max - max(maxima)) < 0.001*abs(max(maxima))) and (result['iteration of d_k_max'] <= winner_iterations):
                winner_parameter = result.method_parameter
                winner_iterations = result['iteration of d_k_max']
        
        parameter_selection.loc[len(parameter_selection)] = subtype, method_name, winner_parameter

parameter_selection.to_csv(filename_parameter_selection, index=False)

In [10]:
parameter_selection

,subtype,method_name,selected_parameter
0,sslp,TA 2,0.01
